## Что нужно сделать?
    (+) переделать получение описаний с запроса к api на поиск по данной html-странице
    (+) обработать все базы и выделить нужные вакансии, перенести в новую базу
    (+) выделить из каждой вакансии список требований
    (+) перенести сюда то, что скинул Саня
    (+) сделать окончательную удобную! базу (id, работодатель, дата публикации, город, требования) 

In [2]:
import sqlite3
import requests
import json
from bs4 import BeautifulSoup

In [104]:
final = 'C:\\Users\\mag\\Desktop\\final.db'

conn = sqlite3.connect(final)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancy')
final_db = cursor.fetchall()
conn.close()

### Создание новой "хорошей" базы данных
    1. используем hh.api для сбора полезных для инфографики характеристик

In [111]:
def collect_data(id_):
    req = requests.get('https://api.hh.ru/vacancies/' + id_).json()
    new_line = (
        id_,
        req['name'],
        req['area']['name'],
        req['employer']['name'],
        req['published_at'],
        req['description']
    )
    return new_line

In [113]:
beautiful_db = []
left = len(final_db)
for job in final_db:
    beautiful_db.append(collect_data(job[0]))
    left -= 1
    if left % 100 == 0:
        print(f'осталось {left} вакансий.')        

осталось 1200 вакансий.
осталось 1100 вакансий.
осталось 1000 вакансий.
осталось 900 вакансий.
осталось 800 вакансий.
осталось 700 вакансий.
осталось 600 вакансий.
осталось 500 вакансий.
осталось 400 вакансий.
осталось 300 вакансий.
осталось 200 вакансий.
осталось 100 вакансий.
осталось 0 вакансий.


In [118]:
#conn = sqlite3.connect('C:\\Users\\mag\\Desktop\\data_set.db')
#cursor = conn.cursor()
#cursor.execute('''CREATE TABLE vacancies (id text, name text, city text, 
#                                            employer text, publication_date text, description text)''')
#for job in beautiful_db:
#    try:
#        cursor.execute(f'''INSERT INTO vacancies VALUES {job}''')
#    except:
#        pass
#conn.commit()

In [122]:
final = 'C:\\Users\\mag\\Desktop\\data_set.db'

conn = sqlite3.connect(final)
cursor = conn.cursor()

cursor.execute('SELECT * from vacancies')
results = cursor.fetchall()
conn.close()

In [171]:
results[1]

('29654641',
 'Frontend разработчик',
 'Москва',
 'Сбербанк',
 '2019-01-15T10:15:40+0300',
 '<p><strong>Требования: </strong></p><p>Опыт web разработки от 3 лет (HTML, HTML5, CSS, JavaScript, препроцессинг CSS)<br />Опыт разработки на библиотеках и фреймворках (AngularJS, ReactJS)<br />Активное использование инструментов контроля версий (Git, CVS)<br />Умение работать с базами данных и языками запросов (SQL)<br />Опыт работы с серверными технологиями (Node.js)<br />Взаимодействие с бекендом (SOAP/REST)</p><div><p></p><p><b>Условия работы:</b></p><p><strong>Мы предлагаем:</strong></p><p> </p><ul><li>трудоустройство согласно ТК РФ;</li><li>гарантированный доход плюс премиальное вознаграждение;</li><li>возможность работать рядом с домом/местом учебы;</li><li>регулярное корпоративное обучение;</li><li>ДМС, страхование от несчастных случаев и тяжелых заболеваний;</li><li>материальную помощь и социальную поддержку, корпоративную пенсионную программу;</li><li>льготные условия кредитования;</l

### Формирование списка всех возможных технических навыков
    1. выкачка с википедии
    2. доп. анализ получившихся требований с целью обнаружения английский названий
    3. добавление аббревиатур

In [125]:
link = 'https://ru.wikipedia.org/wiki/Список_языков_программирования_по_категориям'

In [156]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчщшьыъэюя'

In [134]:
soup = BeautifulSoup(requests.get(link).text, 'lxml')

In [164]:
base = [y for y in [x.text.lower() for x in soup.findAll('li')][20:-67] if set(y) & set(alphabet) == set()]

In [165]:
print(len(base))
base[:5]

161


['aspectc++', 'aspectj', 'caesarj', 'compose*', 'objectteams']

In [169]:
monosyllabic = [x for x in base if len(x.split()) == 1] # односложные названия
monosyllabic[:5]

['aspectc++', 'aspectj', 'caesarj', 'compose*', 'objectteams']